In [27]:
suppressMessages(library(tidyverse))     # tidyverse will pull in ggplot2, readr, other useful libraries
suppressMessages(library(magrittr))      # provides the %>% operator
suppressMessages(library("genefilter"))
suppressMessages(library(DESeq2))
suppressMessages(library(tximport))
suppressMessages(library("readr"))
suppressMessages(library("tximportData"))
suppressMessages(library(data.table))

suppressMessages(library(matrixStats))
suppressMessages(library(org.Hs.eg.db))
suppressMessages(library(topGO))

suppressMessages(library(pheatmap))
suppressMessages(library(gplots))
suppressMessages(library(matrixStats))

SIG_THRESH = 0.01
FC_THRESH = 1.5
WIDTH = 1.6
HEIGHT = 2.6

RES = 300

CELLTYPE_labs = "wbc"

---
### Functions

In [28]:
run_deseq <- function(counts, samples, G1, G2, RUN_TOO = FALSE,CELLTYPES = NULL){
    
    GROUPS = c(G1, G2)
    
    #------------------------------------
    # RUN DESEQ2
    ##------------------------------------

    ##------------------------------------
    # Contstruct DESeq Data Set
    dds <- DESeqDataSetFromMatrix(round(counts),
                                    colData = samples,
                                    design = ~ expGroup + 0)

    ##------------------------------------
    # Add Gene metadata
    annotation = fread(file="../0_support-files/gencode.biotype.name.key.tsv")
    annotation <- annotation[match(rownames(dds), annotation$gene_id),]
    all(rownames(dds) == annotation$ftcount_id)
    mcols(dds) <- cbind(mcols(dds), annotation)


    ##------------------------------------
    # Re-factor
    dds$expGroup <- factor(dds$expGroup, levels = GROUPS)

    ##------------------------------------
    # DAA
    dds <- DESeq(dds)

    ##------------------------------------
    # ANALYZE
    ##------------------------------------

    res <- results(dds,alpha=0.05, contrast = c("expGroup",GROUPS))

    tmp <- res %>% data.frame() %>% filter(abs(log2FoldChange)>1.5) %>% filter(padj < 0.01)
    cat(nrow(tmp))
    
    return(res)
}

run_deseq_wbc <- function(counts, samples, G1, G2, RUN_TOO = FALSE,CELLTYPES = NULL){
    
    GROUPS = c(G1, G2)
    
    #------------------------------------
    # RUN DESEQ2
    ##------------------------------------

    ##------------------------------------
    # Contstruct DESeq Data Set
    dds <- DESeqDataSetFromMatrix(round(counts),
                                    colData = samples,
                                    design = ~ expGroup + wbc + 0)

    ##------------------------------------
    # Add Gene metadata
    annotation = fread(file="../0_support-files/gencode.biotype.name.key.tsv")
    annotation <- annotation[match(rownames(dds), annotation$gene_id),]
    all(rownames(dds) == annotation$ftcount_id)
    mcols(dds) <- cbind(mcols(dds), annotation)


    ##------------------------------------
    # Re-factor
    dds$expGroup <- factor(dds$expGroup, levels = GROUPS)

    ##------------------------------------
    # DAA
    dds <- DESeq(dds)

    ##------------------------------------
    # ANALYZE
    ##------------------------------------

    res <- results(dds,alpha=0.05, contrast = c("expGroup",GROUPS))

    tmp <- res %>% data.frame() %>% filter(abs(log2FoldChange)>1.5) %>% filter(padj < 0.01)
    cat(nrow(tmp))
    
    return(res)
}

In [29]:
create_heatmap <- function(res,groups,cpm,meta_data) {
    
    ANNOTATIONS = c("Diagnosis","severity",'group')
#     ANNOTATIONS = c("Row.names","groups")
    
    ###----------------------------------
    ## Extract significant genes
    sig_genes <- data.frame(res) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% rownames(gene_id)
#             rownames_to_column(var = "gene_id_ensmbl",) %>% mutate(gene_id = gsub(".*_","",gene_id_ensmbl)) %>% pull(gene_id)

    ###----------------------------------
    ## Subset metadata 
    metadata <- meta_data %>% 
        column_to_rownames(var = "Row.names") %>%
        mutate(severity = ifelse(grepl("ontrol",severity),NA,severity)) %>%
        rename(Diagnosis = groups) %>%
        select(all_of(ANNOTATIONS))
    
    colnames(metadata) <- c("Diagnosis","Severity","group")
    
    ###----------------------------------
    ## Subset count matrix
    mat <- data.frame(cpm) %>% 
            filter(row.names(cpm) %in% all_of(sig_genes)) %>% 
            select(all_of(rownames(metadata))) %>% 
            as.matrix()

    mat <- t(scale(t(mat)))

    ###----------------------------------
    ## Colors
    color = colorRampPalette(c("blue","yellow"))(50)
    breaksList = seq(-2, +2, length = 51)

    #color_groups = c('COVID-19\ndiscovery' = '#c1272d', 'MIS-C\ndiscovery' = '#0000a7', 'Control_Non-inflammatory\ndiscovery' = '#eecc16',"MIS-C\nvalidation" = '#008176')
    
    my_colour <- list(
    Diagnosis = c('COV' = '#F0484E', 'M' = '#5CB2EB', 'C' = '#FBE77C') , #"MISC_acute_validation" = '#00FFFF'),
    Severity = c("-1" = "white", "0" = "white", "2" = "#efe5d7", "3" = "#bc8e52"),
    group = c("validation" = "orange", "discovery" = "light blue", "UCSF" = "forest green") #,
    # PCR = c("1" = "green", "2" = "blue"),
    # Antibody =c("1" = "green", "2" = "blue")
    )

    ###----------------------------------
    ## Plot
    
#     return(list(mat, metadata))
    
    # pdf("./tmp.pdf")
    heatmap_plt <- pheatmap(mat,

             # Colors
             col=color,
             breaks=breaksList, 
             annotation_col=metadata,
             annotation_colors=my_colour,
             na_col = "#FFFFFF",

             # Fonts
             show_colnames=F,
             show_rownames=F,
             fontsize=12,
             fontsize_col=3,
             annotation_names_col=F,
             annotation_names_row=F,

             # Clustering

             clustering_distance_cols="correlation",
             clustering_distance_rows="correlation", 
    #          clustering_distance_cols="euclidean",
    #          clustering_distance_rows="euclidean",
    #          cluster_cols=hc,
    #          cluster_rows=hr,
    #          clustering_distance_rows="euclidean",
    #          clustering_method="complete",
             treeheight_row=0,
            treeheight_col= 15,

             # Misc.
             border_color=NA,
            legend=FALSE,
            annotation_legend=FALSE
            ) 
    # dev.off()




    return(heatmap_plt)

        }


---
### read data

In [30]:
labdata <- read.delim("../1_sample-data/wbrna_wbc.tsv") 

labdata %>% head()

,PTID,wbrna_sample_id,wbc
,<chr>,<chr>,<dbl>
1,Patient70,PV92,11.77
2,Patient112,PV26,8.06
3,Patient118,PV28,8.97
4,Patient137,PV58,15.39
5,Patient176,PV220,6.50
6,Patient178,PV222,12.18


---
## COVID19 vs Controls

In [37]:
##------------------------------------
# LOAD METADATA
##------------------------------------

meta_data <- read.csv("../1_sample-data/STable7_wbrna-samples.csv") %>% 
    filter( (Diagnosis == "COVID-19" & timepoint == "acute") | Diagnosis == "Control_Non-inflammatory")

samples <- meta_data

GROUPS = c("COVID-19", "Control_Non-inflammatory")

samples$expGroup <- samples$Diagnosis


table(samples$expGroup)

samples <- merge(samples,labdata, by="wbrna_sample_id",all.x=TRUE)

samples[is.na(samples$wbc),"wbc"] <- mean(samples$wbc, na.rm=T)


##------------------------------------
# PREPARE COUNTS
##------------------------------------

##------------------------------------
# load count data with tximport
sample_ids = unique(samples$wbrna_sample_id)                                                                    # Get sample ids that pass qc

counts = read.delim("../1_sample-data/wbrna_ftcounts.txt")
rownames(counts) <- counts$X

counts = counts[,sample_ids]

##------------------------------------
# Remove ChrX, ChrY, ChrM, and RB genes
gene.list <- read.delim("../0_support-files/genelist.hs.tsv",col.names = c("type,","ENSMBL","gene_symbol"))

gene.ids <- gsub("\\..*","",rownames(counts))

exclude.idx <- gene.ids %in% gene.list[,2]

counts = counts[!exclude.idx,]  

##------------------------------------
# RUN COMPARISON
##------------------------------------

##-------------------------
# Run DESeq2
cat("DEGs w/o CTO: ")
res <- suppressMessages(run_deseq(counts, samples, "COVID-19","Control_Non-inflammatory", FALSE))

cat("\n")

cat("DEGs w/ CTO: ")
res_cto <- suppressMessages(run_deseq_wbc(counts, samples,"COVID-19","Control_Non-inflammatory", TRUE, CELLTYPE_labs))


prefix = "covid_controls_wbc"

write.table(res_cto, paste0("./dea_output/",prefix,".txt"), sep = "\t", quote=FALSE)


Control_Non-inflammatory                 COVID-19 
                      23                       36 

DEGs w/o CTO: 

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


1097
DEGs w/ CTO: 

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


1203

In [1]:
(1203 - 1097) / 1097

[1] 0.09662716

---
## MISC vs Controls

In [13]:
##------------------------------------
# LOAD METADATA
##------------------------------------

meta_data <- read.csv("../1_sample-data/STable7_wbrna-samples.csv") %>% 
    filter( (Diagnosis == "MIS-C" & timepoint == "acute" & group == "discovery") | Diagnosis == "Control_Non-inflammatory")

samples <- meta_data

GROUPS = c("MIS-C", "Control_Non-inflammatory")

samples$expGroup <- samples$Diagnosis


table(samples$expGroup)

samples <- merge(samples,labdata, by="wbrna_sample_id",all.x=TRUE)

samples[is.na(samples$wbc),"wbc"] <- mean(samples$wbc, na.rm=T)


##------------------------------------
# PREPARE COUNTS
##------------------------------------

##------------------------------------
# load count data with tximport
sample_ids = unique(samples$wbrna_sample_id)                                                                    # Get sample ids that pass qc

counts = read.delim("../1_sample-data/wbrna_ftcounts.txt")
rownames(counts) <- counts$X

counts = counts[,sample_ids]

##------------------------------------
# Remove ChrX, ChrY, ChrM, and RB genes
gene.list <- read.delim("../0_support-files/genelist.hs.tsv",col.names = c("type,","ENSMBL","gene_symbol"))

gene.ids <- gsub("\\..*","",rownames(counts))

exclude.idx <- gene.ids %in% gene.list[,2]

counts = counts[!exclude.idx,]  

##------------------------------------
# RUN COMPARISON
##------------------------------------

##-------------------------
# Run DESeq2
cat("DEGs w/o CTO: ")
res <- suppressMessages(run_deseq(counts, samples, "MIS-C","Control_Non-inflammatory", FALSE))

cat("\n")

cat("DEGs w/ CTO: ")
res_cto <- suppressMessages(run_deseq_wbc(counts, samples,"MIS-C","Control_Non-inflammatory", TRUE, CELLTYPE_labs))


prefix = "misc_controls_wbc"

write.table(res_cto, paste0("./dea_output/",prefix,".txt"), sep = "\t", quote=FALSE)


Control_Non-inflammatory                    MIS-C 
                      23                       69 

DEGs w/o CTO: 

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


2024
DEGs w/ CTO: 

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


2093

In [2]:
(2093 - 2024) / 2024

[1] 0.03409091

---
## MISC vs COVID19

In [14]:
##------------------------------------
# LOAD METADATA
##------------------------------------

meta_data <- read.csv("../1_sample-data/STable7_wbrna-samples.csv") %>% 
    filter( (Diagnosis == "MIS-C" & timepoint == "acute" & group == "discovery") |  (Diagnosis == "COVID-19" & timepoint == "acute" & group == "discovery"))

samples <- meta_data

GROUPS = c("MIS-C", "COVID-19")

samples$expGroup <- samples$Diagnosis


table(samples$expGroup)

samples <- merge(samples,labdata, by="wbrna_sample_id",all.x=TRUE)

samples[is.na(samples$wbc),"wbc"] <- mean(samples$wbc, na.rm=T)


##------------------------------------
# PREPARE COUNTS
##------------------------------------

##------------------------------------
# load count data with tximport
sample_ids = unique(samples$wbrna_sample_id)                                                                    # Get sample ids that pass qc

counts = read.delim("../1_sample-data/wbrna_ftcounts.txt")
rownames(counts) <- counts$X

counts = counts[,sample_ids]

##------------------------------------
# Remove ChrX, ChrY, ChrM, and RB genes
gene.list <- read.delim("../0_support-files/genelist.hs.tsv",col.names = c("type,","ENSMBL","gene_symbol"))

gene.ids <- gsub("\\..*","",rownames(counts))

exclude.idx <- gene.ids %in% gene.list[,2]

counts = counts[!exclude.idx,]  

##------------------------------------
# RUN COMPARISON
##------------------------------------

##-------------------------
# Run DESeq2
cat("DEGs w/o CTO: ")
res <- suppressMessages(run_deseq(counts, samples, "MIS-C","COVID-19", FALSE))

cat("\n")

cat("DEGs w/ CTO: ")
res_cto <- suppressMessages(run_deseq_wbc(counts, samples,"MIS-C","COVID-19", TRUE, CELLTYPE_labs))


prefix = "misc_covid_wbc"

write.table(res_cto, paste0("./dea_output/",prefix,".txt"), sep = "\t", quote=FALSE)


COVID-19    MIS-C 
      36       69 

DEGs w/o CTO: 

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


84
DEGs w/ CTO: 

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


92

In [3]:
(92-84)/84

[1] 0.0952381

---
## Pheatmaps

In [2]:
covid_control_labs <- read.delim("./dea_output/covid_controls_wbc.txt")
misc_control_labs <- read.delim("./dea_output/misc_controls_wbc.txt")
misc_covid_labs <- read.delim("./dea_output/misc_covid_wbc.txt")

### Read in metadata
meta_data_all <- read.csv("../1_sample-data/STable7_wbrna-samples.csv")

In [3]:
create_heatmap <- function(res,groups,cpm,meta_data) {
    
    ANNOTATIONS = c("Diagnosis","severity",'group','ivig_rel_samp')

    
    ###----------------------------------
    ## Extract significant genes
    sig_genes <- data.frame(res) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% rownames()

    ###----------------------------------
    ## Subset metadata 
    metadata <- meta_data %>% 
        filter((Diagnosis %in% all_of(groups)) & (timepoint %in% c("Not-hospitalized","acute"))) %>% 
        column_to_rownames(var = "wbrna_sample_id") %>%  
        mutate(severity = ifelse(grepl("ontrol",severity),NA,severity)) %>%
        dplyr::select(all_of(ANNOTATIONS))

    colnames(metadata) <- c("Diagnosis","Severity","group","IVIG")

    ###----------------------------------
    ## Subset count matrix
    mat <- data.frame(cpm) %>% 
            filter(row.names(cpm) %in% all_of(sig_genes)) %>% 
            dplyr::select(all_of(rownames(metadata))) %>% 
            as.matrix()

    mat <- t(scale(t(mat)))

    ###----------------------------------
    ## Colors
    color = colorRampPalette(c("blue","yellow"))(50)
    breaksList = seq(-2, +2, length = 51)

    #color_groups = c('COVID-19\ndiscovery' = '#c1272d', 'MIS-C\ndiscovery' = '#0000a7', 'Control_Non-inflammatory\ndiscovery' = '#eecc16',"MIS-C\nvalidation" = '#008176')
    
    my_colour <- list(
    Diagnosis = c('COVID-19' = '#F0484E', 'MIS-C' = '#5CB2EB', 'Control_Non-inflammatory' = '#FBE77C'), #"MISC_acute_validation" = '#00FFFF'),
    Severity = c("-1" = "white", "0" = "white", "2" = "#efe5d7", "3" = "#bc8e52"),
    group = c("validation" = "orange", "discovery" = "light blue", "UCSF" = "light blue"),
    IVIG = c("after" = "maroon", "before" = "darkseagreen2", "concurrent with" = "dark blue","noivig"="white")#, "forest green"
    # PCR = c("1" = "green", "2" = "blue"),
    # Antibody =c("1" = "green", "2" = "blue")
    )

    ###----------------------------------
    ## Plot

    # pdf("./tmp.pdf")
    heatmap_plt <- pheatmap(mat,

             # Colors
             col=color,
             breaks=breaksList, 
             annotation_col=metadata,
             annotation_colors=my_colour,
             na_col = "#FFFFFF",

             # Fonts
             show_colnames=F,
             show_rownames=F,
             fontsize=12,
             fontsize_col=3,
             annotation_names_col=F,
             annotation_names_row=F,

             # Clustering

             clustering_distance_cols="correlation",
             clustering_distance_rows="correlation", 
    #          clustering_distance_cols="euclidean",
    #          clustering_distance_rows="euclidean",
    #          cluster_cols=hc,
    #          cluster_rows=hr,
    #          clustering_distance_rows="euclidean",
    #          clustering_method="complete",
             treeheight_row=0,
            treeheight_col= 15,

             # Misc.
             border_color=NA,
            legend=FALSE,
            annotation_legend=FALSE
            ) 
    # dev.off()




    return(heatmap_plt)

        }

In [4]:
##------------------------------------
# Read ftcount matrix
raw_ftcount <- read.delim("../1_sample-data/wbrna_ftcounts.txt",row.names = 1)
nrow(raw_ftcount)

##------------------------------------
# Remove ChrX, ChrY, ChrM, and RB genes
gene.list <- read.delim("../0_support-files/genelist.hs.tsv",col.names = c("type,","ENSMBL","gene_symbol"))

gene.ids <- gsub("\\..*","",rownames(raw_ftcount))

exclude.idx <- as.character(gene.ids) %in% as.character(gene.list[,2])

raw_ftcount = raw_ftcount[!exclude.idx,]  

##------------------------------------
# subset
raw_ftcount <- raw_ftcount[,colSums(raw_ftcount) > 0]

cpm_ftcount <- edgeR::cpm(raw_ftcount)

nrow(cpm_ftcount)

[1] 60708

[1] 56634

In [5]:
SIG_THRESH = 0.01
FC_THRESH = 1.5

# SIG_THRESH = 0.05
# FC_THRESH = 0

ANNOTATIONS <- c("Diagnosis","severity")

WIDTH = 1.6
HEIGHT = 2.6

RES = 300

## Make the heatmaps

In [9]:
covid_control_labs_plt <- create_heatmap(covid_control_labs,c("COVID-19","Control_Non-inflammatory"),cpm_ftcount,meta_data_all)
misc_control_labs_plt <- create_heatmap(misc_control_labs,c("MIS-C","Control_Non-inflammatory"),cpm_ftcount,meta_data_all)
misc_covid_labs_plt <- create_heatmap(misc_covid_labs,c("MIS-C","COVID-19"),cpm_ftcount,meta_data_all)

`use_raster` is automatically set to TRUE for a matrix with more than
2000 rows. You can control `use_raster` argument by explicitly setting
TRUE/FALSE to it.

Set `ht_opt$message = FALSE` to turn off this message.

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.



In [7]:
suppressMessages(require(pheatmap))
suppressMessages(require(RColorBrewer))
suppressMessages(require(dendextend))
suppressMessages(require(ComplexHeatmap))
suppressMessages(require(circlize))



In [10]:
##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "covid-control"
       
png(file=paste0("plots/SupPanelE_",prefix,".png"),
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(covid_control_labs_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "misc-control"

png(file=paste0("plots/SupPanelE_",prefix,".png"),
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(misc_control_labs_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "misc-covid"

png(file=paste0("plots/SupPanelE_",prefix,".png"),
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(misc_covid_labs_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

png 
  2

png 
  2

png 
  2